<a href="https://colab.research.google.com/github/v5v5/trading/blob/main/trade_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

URL to download quotes EURUSD

http://export.finam.ru/EURUSD_210101_210425.csv?market=1&em=175924&code=EURUSD&apply=0&df=1&mf=1&yf=2021&from=01.01.2021&dt=25&mt=4&yt=2021&to=25.04.2021&p=8&f=EURUSD_210101_210425&e=.csv&cn=EURUSD&dtf=1&tmf=1&MSOR=0&mstime=on&mstimever=1&sep=3&sep2=1&datf=1&at=1

In [15]:
import pandas as pd
import urllib.request
from io import StringIO

In [16]:
url = 'http://export.finam.ru/EURUSD_210101_210425.csv?market=1&em=175924&code=EURUSD&apply=0&df=1&mf=1&yf=2021&from=01.01.2021&dt=25&mt=4&yt=2021&to=25.04.2021&p=8&f=EURUSD_210101_210425&e=.csv&cn=EURUSD&dtf=1&tmf=1&MSOR=0&mstime=on&mstimever=1&sep=3&sep2=1&datf=1&at=1'


In [17]:
page = urllib.request.urlopen(url)
content = page.read()

In [18]:
text = content.decode('UTF-8')

In [19]:
print(text[:500])

<TICKER>;<PER>;<DATE>;<TIME>;<OPEN>;<HIGH>;<LOW>;<CLOSE>;<VOL>
EURUSD;D;20210201;000000;1690.5000000;1778.0000000;1671.8000000;1746.5000000;5472389
EURUSD;D;20210202;000000;1739.9000000;1739.9000000;1693.2000000;1712.0000000;2314296
EURUSD;D;20210203;000000;1713.2000000;1739.0000000;1695.0000000;1707.0000000;1214799
EURUSD;D;20210204;000000;1694.9000000;1702.3000000;1640.1000000;1670.8000000;2473501
EURUSD;D;20210205;000000;1667.4000000;1681.2000000;1653.4000000;1675.0000000;1504035
EURUSD


In [20]:
data = StringIO(text)

In [21]:
df1 = pd.read_csv(data, sep=';', header='infer')

In [22]:
df1.head()

,<TICKER>,<PER>,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,EURUSD,D,20210201,0,1690.5,1778.0,1671.8,1746.5,5472389
1,EURUSD,D,20210202,0,1739.9,1739.9,1693.2,1712.0,2314296
2,EURUSD,D,20210203,0,1713.2,1739.0,1695.0,1707.0,1214799
3,EURUSD,D,20210204,0,1694.9,1702.3,1640.1,1670.8,2473501
4,EURUSD,D,20210205,0,1667.4,1681.2,1653.4,1675.0,1504035
